# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 31 2023 1:14PM,256311,10,0086342341,ISDIN Corporation,Released
1,Jan 31 2023 1:14PM,256311,10,0086342342,ISDIN Corporation,Released
2,Jan 31 2023 1:14PM,256311,10,0086342345,ISDIN Corporation,Released
3,Jan 31 2023 1:14PM,256311,10,0086342346,ISDIN Corporation,Released
4,Jan 31 2023 1:14PM,256311,10,0086342343,ISDIN Corporation,Released
5,Jan 31 2023 1:14PM,256311,10,0086342347,ISDIN Corporation,Released
6,Jan 31 2023 1:14PM,256311,10,0086342344,ISDIN Corporation,Released
7,Jan 31 2023 1:14PM,256311,10,0086342355,ISDIN Corporation,Released
8,Jan 31 2023 1:14PM,256311,10,0086342359,ISDIN Corporation,Released
9,Jan 31 2023 1:14PM,256311,10,0086342358,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,256308,Released,1
50,256309,Released,1
51,256310,Released,10
52,256311,Released,21
53,256312,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256308,NaN,NaN,1.0
256309,NaN,NaN,1.0
256310,NaN,NaN,10.0
256311,NaN,NaN,21.0
256312,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256218,0.0,0.0,1.0
256236,1.0,26.0,2.0
256240,10.0,15.0,12.0
256245,0.0,0.0,1.0
256252,0.0,1.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256218,0,0,1
256236,1,26,2
256240,10,15,12
256245,0,0,1
256252,0,1,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256218,0,0,1
1,256236,1,26,2
2,256240,10,15,12
3,256245,0,0,1
4,256252,0,1,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256218,,,1
1,256236,1,26,2
2,256240,10,15,12
3,256245,,,1
4,256252,,1,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation
21,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation
22,Jan 31 2023 1:13PM,256310,10,"Methapharm, Inc."
32,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc."
33,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions
34,Jan 31 2023 12:56PM,256307,12,"Uniderm USA, Inc"
35,Jan 31 2023 12:52PM,256303,12,"Uniderm USA, Inc"
36,Jan 31 2023 12:46PM,256302,19,"Emersa Waterbox, LLC"
37,Jan 31 2023 12:37PM,256300,20,"HVL, LLC dba Atrium Innovations"
38,Jan 31 2023 12:32PM,256299,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,,,21
1,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,,,1
2,Jan 31 2023 1:13PM,256310,10,"Methapharm, Inc.",,,10
3,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",,,1
4,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,,,1
5,Jan 31 2023 12:56PM,256307,12,"Uniderm USA, Inc",,,1
6,Jan 31 2023 12:52PM,256303,12,"Uniderm USA, Inc",,,1
7,Jan 31 2023 12:46PM,256302,19,"Emersa Waterbox, LLC",,,1
8,Jan 31 2023 12:37PM,256300,20,"HVL, LLC dba Atrium Innovations",,,1
9,Jan 31 2023 12:32PM,256299,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,21,,
1,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,1,,
2,Jan 31 2023 1:13PM,256310,10,"Methapharm, Inc.",10,,
3,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",1,,
4,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,1,,
5,Jan 31 2023 12:56PM,256307,12,"Uniderm USA, Inc",1,,
6,Jan 31 2023 12:52PM,256303,12,"Uniderm USA, Inc",1,,
7,Jan 31 2023 12:46PM,256302,19,"Emersa Waterbox, LLC",1,,
8,Jan 31 2023 12:37PM,256300,20,"HVL, LLC dba Atrium Innovations",1,,
9,Jan 31 2023 12:32PM,256299,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,21,,
1,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,1,,
2,Jan 31 2023 1:13PM,256310,10,"Methapharm, Inc.",10,,
3,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",1,,
4,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,21.0,NaN,NaN
1,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,1.0,NaN,NaN
2,Jan 31 2023 1:13PM,256310,10,"Methapharm, Inc.",10.0,NaN,NaN
3,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,21.0,0.0,0.0
1,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,1.0,0.0,0.0
2,Jan 31 2023 1:13PM,256310,10,"Methapharm, Inc.",10.0,0.0,0.0
3,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6407134,170.0,23.0,3.0
12,512610,2.0,0.0,0.0
15,768776,65.0,65.0,10.0
16,1025191,5.0,1.0,0.0
18,256276,1.0,0.0,0.0
19,1537620,11.0,27.0,1.0
20,256300,1.0,0.0,0.0
21,256218,1.0,0.0,0.0
22,256245,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6407134,170.0,23.0,3.0
1,12,512610,2.0,0.0,0.0
2,15,768776,65.0,65.0,10.0
3,16,1025191,5.0,1.0,0.0
4,18,256276,1.0,0.0,0.0
5,19,1537620,11.0,27.0,1.0
6,20,256300,1.0,0.0,0.0
7,21,256218,1.0,0.0,0.0
8,22,256245,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,170.0,23.0,3.0
1,12,2.0,0.0,0.0
2,15,65.0,65.0,10.0
3,16,5.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,11.0,27.0,1.0
6,20,1.0,0.0,0.0
7,21,1.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,170.0
1,12,Released,2.0
2,15,Released,65.0
3,16,Released,5.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,3.0,0.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0
Executing,23.0,0.0,65.0,1.0,0.0,27.0,0.0,0.0,0.0
Released,170.0,2.0,65.0,5.0,1.0,11.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,3.0,0.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,23.0,0.0,65.0,1.0,0.0,27.0,0.0,0.0,0.0
2,Released,170.0,2.0,65.0,5.0,1.0,11.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,3.0,0.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,23.0,0.0,65.0,1.0,0.0,27.0,0.0,0.0,0.0
2,Released,170.0,2.0,65.0,5.0,1.0,11.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()